# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import api_key
from configs import g_keys

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/City_Data.csv")
weather_data

,city,country,humidity,temp,cloud,windy,lat,lng
0,Mazagão,BR,74,86.00,20,9.17,-0.12,-51.29
1,Ushuaia,AR,86,33.80,75,26.40,-54.80,-68.30
2,Atuona,PF,79,77.56,30,24.29,-9.80,-139.03
3,George,ZA,93,48.20,18,3.36,-33.96,22.46
4,Tasiilaq,GL,80,35.60,0,1.12,65.61,-37.64
...,...,...,...,...,...,...,...,...
551,Petropavlovsk-Kamchatskiy,RU,87,53.60,100,8.95,53.05,158.65
552,Barrow,US,87,46.40,90,9.17,71.29,-156.79
553,Saint Paul Harbor,US,62,55.40,90,9.17,57.79,-152.41
554,New Norfolk,AU,67,41.79,0,1.01,-42.78,147.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_keys)

# Store latitude and longitude in locations
locations = weather_data[["lat", "lng"]]

# Store Humidity in humidity
humidity = weather_data["humidity"]

In [5]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Find median coordinates
mid_lat = weather_data["lat"].median()
mid_long = weather_data["lng"].median()
center = [mid_lat, mid_long]

#Create Figure
fig = gmaps.figure(zoom_level=2, center = center)

# Add heatmap layer to figure
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the cities with wind speed less than 35 mph, cloudiness equals to 0 and max temp between 60 and 75
narrowed_city_df = weather_data.loc[(weather_data["windy"] <= 35) & (weather_data["cloud"] == 20) & \
                                   (weather_data["temp"] >= 60) & (weather_data["temp"] <= 75)].dropna()

narrowed_city_df

#For some reason Jamestown gets randomly created a metric shit ton of times
#Quite a random set of numbers there

,city,country,humidity,temp,cloud,windy,lat,lng
41,Chlórakas,CY,83,71.76,20,4.70,34.80,32.41
59,Ponta do Sol,PT,64,65.97,20,6.93,32.67,-17.10
72,Jamestown,US,72,62.55,20,5.82,42.10,-79.24
124,Camacha,PT,68,71.60,20,23.04,33.08,-16.33
127,Jamestown,US,72,62.55,20,5.82,42.10,-79.24
145,Tonekābon,IR,77,69.80,20,2.24,36.82,50.87
158,Jamestown,US,72,62.55,20,5.82,42.10,-79.24
233,Ponta do Sol,PT,64,65.97,20,6.93,32.67,-17.10
236,Melilla,ES,94,72.73,20,4.70,35.29,-2.94
237,Jamestown,US,72,62.55,20,5.82,42.10,-79.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["city","country", "lat", "lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = "Hyatt"

# Display the result
hotel_df

,city,country,lat,lng,Hotel Name
41,Chlórakas,CY,34.80,32.41,Hyatt
59,Ponta do Sol,PT,32.67,-17.10,Hyatt
72,Jamestown,US,42.10,-79.24,Hyatt
124,Camacha,PT,33.08,-16.33,Hyatt
127,Jamestown,US,42.10,-79.24,Hyatt
145,Tonekābon,IR,36.82,50.87,Hyatt
158,Jamestown,US,42.10,-79.24,Hyatt
233,Ponta do Sol,PT,32.67,-17.10,Hyatt
236,Melilla,ES,35.29,-2.94,Hyatt
237,Jamestown,US,42.10,-79.24,Hyatt


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_keys}

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]
print(locations)

#had to touch your code

       lat     lng
41   34.80   32.41
59   32.67  -17.10
72   42.10  -79.24
124  33.08  -16.33
127  42.10  -79.24
145  36.82   50.87
158  42.10  -79.24
233  32.67  -17.10
236  35.29   -2.94
237  42.10  -79.24
261  42.10  -79.24
273  39.30    9.20
294  42.10  -79.24
364  32.67  -17.10
366  38.73  -27.07
374  40.60 -124.16
405  42.10  -79.24
487  42.10  -79.24


In [14]:
#Format map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…